In [1]:
import numpy as np
import sympy as sym
import numba
from scipy.integrate import ode
from scipy import optimize
from collections import deque 
import pydae.build as db

In [2]:
R = 0.1  # Resistor (Ohm)
L = 0.01 # Inductor (L)

# Control design
T_pi = L/R    # PI time constant
K_p = L/0.01  # PI proportational gain

params= {'R' : 0.1, 'L':0.01, 'V_max':1.0, 'V_min':0.0, 'K_p':K_p, 'T_pi':T_pi, 'K_aw':1.0}
u_ini_dict = { 'i_ref':1.0}  # for the initialization problem
u_run_dict = { 'i_ref':1.0}  # for the running problem (here initialization and running problem are the same)


x_list = ['i','xi']    # [inductor current, PI integrator]
y_ini_list = ['y','v'] # for the initialization problem
y_run_list = ['y','v'] # for the running problem (here initialization and running problem are the same)

sys_vars = {'params':params,
            'u_list':u_run_dict,
            'x_list':x_list,
            'y_list':y_run_list}


In [3]:
exec(db.sym_gen_str())  # exec to generate the required symbolic varables and constants

In [7]:
epsilon = i_ref - i

di  = 1/L*(v-i*R)
dxi = epsilon - K_aw*(y - v)             

g_1 = -y + K_p*(epsilon +1/T_pi*xi)
g_2 = -v + sym.Piecewise((V_min,y<V_min),(V_max,y>V_max),(y,True))

h_1 = v

x = sym.Matrix(x_list)

sys = {'name':'rl_ctrl_sat_aw',
       'params_dict':params,
       'f_list':[di,dxi],
       'g_list':[g_1,g_2],
       'x_list':x_list,
       'y_ini_list':y_ini_list,
       'y_run_list':y_run_list,
       'u_run_dict':u_run_dict,
       'u_ini_dict':u_ini_dict,
       'h_dict':{'h_1':h_1}}

sys = db.system(sys)
db.sys2num(sys)